# Keras Tutorail - the Happy House

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *
from keras.models import Sequential

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


## 1. Dataset

In [3]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


## 2. Building model in Keras

In [5]:
def happy_model(input_shape):
    x_input = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(x_input)
    X = Conv2D(32, (7,7), strides=(1, 1),name = "conv0")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    X = Dropout(0.25)(X)
    X = Conv2D(32, (7,7), strides=(1, 1),name = "conv1")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool2')(X)
    
    X = Flatten()(X)
    X = Dense(128)(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    model = Model(inputs=x_input,outputs=X,name="HappyModel")
    
    return model

## 3. Train & test

In [6]:
happyModel = happy_model(X_train.shape[1:])

In [7]:
happyModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [8]:
happyModel.fit(x=X_train,y=Y_train,batch_size=32,epochs=10)

Epoch 1/10
600/600 [==============================] - 5s 9ms/step - loss: 0.9137 - acc: 0.5250
Epoch 2/10
600/600 [==============================] - 5s 8ms/step - loss: 0.6261 - acc: 0.6417
Epoch 3/10
600/600 [==============================] - 5s 8ms/step - loss: 0.5101 - acc: 0.7383
Epoch 4/10
600/600 [==============================] - 5s 8ms/step - loss: 0.3899 - acc: 0.8183
Epoch 5/10
600/600 [==============================] - 5s 8ms/step - loss: 0.3290 - acc: 0.8600
Epoch 6/10
600/600 [==============================] - 5s 8ms/step - loss: 0.2851 - acc: 0.8667
Epoch 7/10
600/600 [==============================] - 5s 8ms/step - loss: 0.2528 - acc: 0.9050
Epoch 8/10
600/600 [==============================] - 5s 8ms/step - loss: 0.2807 - acc: 0.8717
Epoch 9/10
600/600 [==============================] - 5s 8ms/step - loss: 0.2363 - acc: 0.9050
Epoch 10/10
600/600 [==============================] - 5s 8ms/step - loss: 0.1891 - acc: 0.9283


In [9]:
preds = happyModel.evaluate(X_test,Y_test,batch_size=32,verbose=1,sample_weight=None)

print("Loss = ",str(preds[0]))
print("Test accuracy = ",str(preds[1]))

150/150 [==============================] - 1s 4ms/step
Loss =  0.207613831361
Test accuracy =  0.92666667064


## 4. Model debug

In [10]:
happyModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 26, 26, 32)        50208     
__________

### Calculating number of weights for Con layer
-------------------------
[Calculate # of weights](https://stackoverflow.com/questions/44608552/keras-cnn-model-parameters-calculation)

    - In this formula '+1' is because of bias for per filter

In [ ]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))